In [2]:
pip install kiteconnect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 771.5/771.5 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 113.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.7/264.7 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 5.5 MB/s eta 0:00:00


In [3]:
import logging
from kiteconnect import KiteConnect

logging.basicConfig(level=logging.DEBUG)

kite = KiteConnect(api_key="vsvq1sfx668831uz")

In [4]:
kite.login_url()

'https://kite.zerodha.com/connect/login?api_key=vsvq1sfx668831uz&v=3'

In [5]:
data = kite.generate_session("2CcpvCHqGLuY4swzh1ywL1amwnHCIyrd",api_secret="n63k0cfcuyxd5k8dis5zzkzpkyybxkhn")
kite.set_access_token(data["access_token"])

In [10]:
import pandas as pd
from datetime import datetime, timedelta
instrument_token = 256265  # NIFTY 50 spot
to_date = datetime.now()
from_date = to_date - timedelta(days=365)
interval="day"
data = kite.historical_data(
        instrument_token=instrument_token,
        from_date=from_date,
        to_date=to_date,
        interval=interval
    )
df = pd.DataFrame(data)
df.rename(columns={'date': 'timestamp'}, inplace=True)

In [12]:
df = df.copy()
df['SMA_short'] = df['close'].rolling(window=short_window).mean()
df['SMA_long'] = df['close'].rolling(window=long_window).mean()

# Trading signals
df['signal'] = 0
df.loc[df['SMA_short'] > df['SMA_long'], 'signal'] = 1
df.loc[df['SMA_short'] < df['SMA_long'], 'signal'] = -1

# Strategy returns
df['daily_return'] = df['close'].pct_change()
df['strategy_return'] = df['signal'].shift(1) * df['daily_return']

total_return = (df['strategy_return'] + 1).prod() - 1
sharpe_ratio = (
      df['strategy_return'].mean() / df['strategy_return'].std()
) * np.sqrt(252)

results = {
        "Total Return (%)": round(total_return * 100, 2),
        "Sharpe Ratio": round(sharpe_ratio, 2),
    }

if plot:
      plt.figure(figsize=(12, 6))
      (df['strategy_return'].cumsum()).apply(np.exp).plot(label='Strategy', color='blue')
      (df['daily_return'].cumsum()).apply(np.exp).plot(label='Buy & Hold', color='gray')
      plt.legend()
      plt.title("Moving Average Crossover Backtest")
      plt.grid(True)
      plt.show()

,timestamp,open,high,low,close,volume
243,2025-10-30 00:00:00+05:30,25984.40,26032.05,25845.25,25877.85,0
244,2025-10-31 00:00:00+05:30,25863.80,25953.75,25711.20,25722.10,0
245,2025-11-03 00:00:00+05:30,25696.85,25803.10,25645.50,25763.35,0
246,2025-11-04 00:00:00+05:30,25744.75,25787.40,25578.40,25597.65,0
247,2025-11-06 00:00:00+05:30,25593.35,25679.15,25520.75,25550.75,0
